In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
Irish = np.load('Data_from_Ulrichs_paper/Irish_data.npz')

data = [Irish['freq_model1'],Irish['freq_model3'],Irish['freq_model4']]
# Remove NaN values from the data
data = [d[~np.isnan(d)] for d in data]

# Define the regions dictionary
region_dict = {0: 'Model 1', 1: 'Model 3', 2: 'Model 4'}
colors=('#1b9e77', '#d95f02','#7570b3')

In [20]:
def LTtest (data):
    # Compute Fourier transform 
    fft_0 = np.fft.fft(data)
    
    # Randomize phases of Fourier coefficients
    rand_phases = np.random.uniform(0, 2*np.pi, size=len(fft_0))
    surrogate = np.abs(fft_0) * np.exp(1j * rand_phases)
    
    # Compute inverse Fourier transform to obtain surrogate data
    surrogate = np.real(np.fft.ifft(surrogate))
    
    data = data[~np.isnan(data)]
    L = len(data)
    
    tau = np.arange(1, 82801,540 )
    #tau = np.arange(0, L//2, 3600)
    res_1 = np.zeros(len(tau))
    res_2 = np.zeros(len(tau))
    surr_1 = np.zeros(len(tau))
    surr_2 = np.zeros(len(tau))

    for i in range(len(tau)):
        x_t = data[0 :L-tau[i]]
        x_tau = data[tau[i]:L]
        y_t = surrogate[0:(L-tau[i])]
        y_tau = surrogate[(tau[i]):L]

        # Second method LT2
        res_2[i] = np.mean((x_t-x_tau)**3)/np.mean((x_t-x_tau)**2)
        surr_2[i] = np.mean((y_t-y_tau)**3)/np.mean((y_t-y_tau)**2)      
        
    # Calculate the rmse(LT2)
    mse_lt2 = mean_squared_error(np.nan_to_num(res_2), np.nan_to_num(surr_2))
    rmse_lt2 = np.sqrt(mse_lt2)
    
    #return rmse_lt2
    return rmse_lt2

In [21]:
model_lt = []
for i, df in enumerate(data):
    rmse_lt2 = LTtest(df)
    model_lt.append((region_dict[i], rmse_lt2))
#%store euro_lt

In [23]:
%store model_lt

Stored 'model_lt' (list)
